In [1]:
!pip install --upgrade pythainlp
!pip install emoji

     |████████████████████████████████| 8.9MB 4.3MB/s 
     |████████████████████████████████| 747kB 45.4MB/s 
     |████████████████████████████████| 51kB 2.8MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=47271ec389debee2a70292e09a78ab0adf6c46c096a03627eebd0d1fd74aeeb7
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
from pythainlp.corpus.common import thai_stopwords
from pythainlp import sent_tokenize, word_tokenize
from pythainlp.ulmfit import *
from pythainlp.util import normalize
import re
import itertools
import dill as pickle

Corpus: wiki_lm_lstm
- Downloading: wiki_lm_lstm 0.32


100%|██████████| 1050919089/1050919089 [00:11<00:00, 87620291.53it/s]


Corpus: wiki_itos_lstm
- Downloading: wiki_itos_lstm 0.32


100%|██████████| 1530484/1530484 [00:00<00:00, 17898151.34it/s]


Text cleaning

In [3]:
def text_cleaning(texts):
  clean_word = []
  stop_words = thai_stopwords() 
  for text in texts:
    #emoji list
    pos_emoji = re.compile(u'[\U0001F600\U0001F603\U0001F604\U0001F601\U0001F606\U0001F60A\U0000263A\U0000FE0F\U0001F923\U0001F642\U0001F609\U0001F60C\U0001F619\U0001F617\U0001F618\U0001F970\U0001F60D\U0001F61A\U0001F60B\U0001F61B\U0001F61D\U0001F61C\U0001F973\U0001F60F\U0001F633\U0001F638\U0001F63A\U0001F63D\U0001F63B\U0001F63C\U0001F44D\U0001F3FB\U0001F91F\U0001F3FB\U0001F918\U0001F3FB\U0001F48B\U00002764\U0000FE0F\U0001F9E1\U0001F49B\U0001F49A\U0001F499\U0001F49C\U00002763\U0000FE0F\U0001F495\U0001F49E\U0001F493\U0001F497\U0001F496\U0001F498\U0001F49D]',
            flags=re.UNICODE)
    neg_emoji = re.compile(u'[\U0001F494\U0001F642\U0001F643\U0001F61E\U0001F612\U0001F60F\U0001F614\U0001F61F\U0001F615\U0001F641\U00002639\U0000FE0F\U0001F623\U0001F616\U0001F62B\U0001F629\U0001F97A\U0001F622\U0001F62D\U0001F60F\U0001F624\U0001F620\U0001F621\U0001F92C\U0001F92F\U0001F975\U0001F628\U0001F630\U0001F625\U0001F613\U0001F925\U0001F636\U0001F610\U0001F611\U0001F644\U0001F626\U0001F640\U0001F63E\U0001F63C\U0001F595\U0001F3FB\U0001F44E\U0001F3FB\U0001F9B6\U0001F3FB\U0001F448\U0001F3FB\U0001F91E\U0001F3FB\U0001F44B\U0001F3FB\U0001F47F\U0001F47A\U0001F921\U0001F92E\U0001F974\U0001F463]',
            flags=re.UNICODE)
    pos_count = len(re.findall(pos_emoji,text))
    neg_count = len(re.findall(neg_emoji,text))
    #text.replace('☺️', 'posemo')
    #for emo in pos_emoji: text = text.replace(emo,'posemo')

    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002500-\U00002BEF"  # chinese char
                                u"\U00002702-\U000027B0"
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                u"\U0001f926-\U0001f937"
                                u"\U00010000-\U0010ffff"
                                u"\u2640-\u2642"
                                u"\u2600-\u2B55"
                                u"\u200d"
                                u"\u23cf"
                                u"\u23e9"
                                u"\u231a"
                                u"\ufe0f"  # dingbats
                                u"\u3030"
                                "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r"", text)
    #delte Link hashtag and mention
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "",text)
    text = re.sub(r"^https://t.co/[A-Za-z0-9]*\s","",text)
    text = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", "", text)
    text = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", "", text)
    #find and delete laugh
    laugh_count = len(re.findall(r'(5)\1{2,}(6?){3,}', text))
    text = re.sub(r'(5)\1{2,}(6?){3,}', '', text)
    #delete symbol
    text = re.sub(r'[!-@[-`{-~]',"",text)
    #text = re.sub("\d+", "", text) #number
    text = normalize(text)

    
#Tokenization

    tokens = process_thai(text)

#deletion of whitespace & one letter text

    i = 0;
    for token in list(tokens):
      if(len(token) == 1 or len(token) == token.count(token[0]) or token in ['xxrep','xxwrep','','ชา','นนท์','ปอนด์','ป้อม']):
        tokens.pop(i)
        i = i-1
      i = i+1

#Add thailaugh posemoji negemoji tag

    for a in range(laugh_count):
      tokens.append('thailaugh')
    for a in range(pos_count):
      tokens.append('posemoji')
    for a in range(neg_count):
      tokens.append('negemoji')
    
# POS Tag

    # from pythainlp.tag import pos_tag
    # pos = pos_tag(tokens,corpus='orchid_ud')
    # keep_tag = ['VERB', 'ADJ', 'ADV', 'INTJ', 'AUX']
    #keep_tag = ['VACT','VATT','ADVN','ADVI','ADVP','ADVS','FIXV','NEG','ADJ','']
    
    
    # pos_tags = [t[0] for t in pos if (t[1] in keep_tag) or (t[0] == "thailaugh") 
    # or (t[0] == "posemoji")  or (t[0] == "negemoji")]
    # tokens = pos_tags

# Delete Stop Word

    filtered_sentence = []
    for t in tokens: 
      if t not in stop_words:
        #t = ''.join(c[0] for c in itertools.groupby(t))
        filtered_sentence.append(t)
    clean_word.append(','.join(filtered_sentence))
  return clean_word

importing data and model

In [4]:
clf = pickle.load(open("/content/drive/My Drive/ICT senior project/model/clf", "rb"))
tfidf = pickle.load(open("/content/drive/My Drive/ICT senior project/tfidf/tfidf.pickle","rb"))
feature = pickle.load(open("/content/drive/My Drive/ICT senior project/tfidf/feature.pickle","rb"))

In [5]:
predict = pd.read_csv('/content/drive/My Drive/ICT senior project/masterchef-dataset.csv')
predict = predict[4900:]
predict.head()

,text_id,sentiment,text,date
4900,4901,NaN,หง่าาาาา เชฟพฤกษ์คือ ละมุนมากกกกกก #MasterChef...,2020-03-22 12:30:55
4901,4902,NaN,เควสจ๋า ห้ามออก!!!!!!!!!!!!!! #MasterChefAllS...,2020-03-22 12:30:55
4902,4903,NaN,เชฟพฤกษ์มาเป็นพิธีกรเพิ่มด้วยได้ไหมคะ ละมุน อบ...,2020-03-22 12:30:55
4903,4904,NaN,มีใครเวลากินมันฝรั่งเเล้วต้องดูว่าแต่ละชิ้นมัน...,2020-03-22 12:30:54
4904,4905,NaN,เชฟพฤกษ์คือสุภาพมากกกกกกกกก #MasterChefAllStar...,2020-03-22 12:30:54


In [6]:
predict.reset_index()['text']

0     หง่าาาาา เชฟพฤกษ์คือ ละมุนมากกกกกก #MasterChef...
1     เควสจ๋า ห้ามออก!!!!!!!!!!!!!!  #MasterChefAllS...
2     เชฟพฤกษ์มาเป็นพิธีกรเพิ่มด้วยได้ไหมคะ ละมุน อบ...
3     มีใครเวลากินมันฝรั่งเเล้วต้องดูว่าแต่ละชิ้นมัน...
4     เชฟพฤกษ์คือสุภาพมากกกกกกกกก #MasterChefAllStar...
                            ...                        
95    กูไม่อยากให้ใครอ๊อกกกกกกกกกก!!! #MasterChefAll...
96                  จ๋าสู้ๆ #MasterChefAllStarsThailand
97    จะรอดมั้ยวะกู#ทีมชานูปปี้ #MasterchefAllStarsT...
98        พี่ปอนด์หนูรักพี่ #MasterChefAllStarsThailand
99                 ไม่นะๆๆๆ #MasterChefAllStarsThailand
Name: text, Length: 100, dtype: object

In [7]:
clean_list = text_cleaning(predict['text'])
b_a = pd.DataFrame(predict.reset_index()['text'])
b_a['after'] = clean_list
b_a = b_a.rename(columns={'text':'before'})
b_a.head(10)

,before,after
0,หง่าาาาา เชฟพฤกษ์คือ ละมุนมากกกกกก #MasterChef...,"ง่า,เชฟ,พฤกษ์,ละมุน"
1,เควสจ๋า ห้ามออก!!!!!!!!!!!!!! #MasterChefAllS...,"เควส,ห้าม"
2,เชฟพฤกษ์มาเป็นพิธีกรเพิ่มด้วยได้ไหมคะ ละมุน อบ...,"เชฟ,พฤกษ์,พิธีกร,ไหม,ละมุน,อบอุ่น,แง"
3,มีใครเวลากินมันฝรั่งเเล้วต้องดูว่าแต่ละชิ้นมัน...,"เวลา,กิน,มันฝรั่ง,ดู,ชิ้น,หรอ,ตั่ยๆ"
4,เชฟพฤกษ์คือสุภาพมากกกกกกกกก #MasterChefAllStar...,"เชฟ,พฤกษ์,สุภาพ"
5,ไม่อยากให้ใครออกเลย😭😭😭 #MasterChefAllStarsThai...,"negemoji,negemoji,negemoji"
6,อยากให้เชฟพฤกษ์มาทุกวีค 55555 #MasterChefAllSt...,"เชฟ,พฤกษ์,วีค,thailaugh"
7,เชฟพฤกษ์โหดมาก #MasterChefAllStarsThailand,"เชฟ,พฤกษ์,โหด"
8,ตอนแรกเชฟห่วงชานนท์ ต่อมาห่วงปอนด์ ต่อมาห่วงจ๋...,"ตอนแรก,เชฟ,ห่วง,ต่อมา,ห่วง,ต่อมา,ห่วง,เข้ามา,เ..."
9,เชฟแด๊ดดี๊ ฮืออ หลง #MasterChefAllStarsThailand,"เชฟแด๊ดดี๊,ฮือ,หลง"


In [8]:
term_weighting = tfidf.transform(clean_list)

In [9]:
term_weighting.shape

(100, 4478)

In [10]:
predict_feature = [n[feature] for n in term_weighting.toarray()]

In [13]:
new_list = []
for i in range(len(clean_list)):
  new_list.append([predict['text'][i+4900], clf.predict(predict_feature)[i]])
b_a = pd.DataFrame(new_list, columns=['text', 'predict'])

#pd.DataFrame(clf.predict(predict_feature)

In [16]:
b_a.head(10)

,text,predict
0,หง่าาาาา เชฟพฤกษ์คือ ละมุนมากกกกกก #MasterChef...,2.0
1,เควสจ๋า ห้ามออก!!!!!!!!!!!!!! #MasterChefAllS...,1.0
2,เชฟพฤกษ์มาเป็นพิธีกรเพิ่มด้วยได้ไหมคะ ละมุน อบ...,2.0
3,มีใครเวลากินมันฝรั่งเเล้วต้องดูว่าแต่ละชิ้นมัน...,1.0
4,เชฟพฤกษ์คือสุภาพมากกกกกกกกก #MasterChefAllStar...,2.0
5,ไม่อยากให้ใครออกเลย😭😭😭 #MasterChefAllStarsThai...,1.0
6,อยากให้เชฟพฤกษ์มาทุกวีค 55555 #MasterChefAllSt...,2.0
7,เชฟพฤกษ์โหดมาก #MasterChefAllStarsThailand,2.0
8,ตอนแรกเชฟห่วงชานนท์ ต่อมาห่วงปอนด์ ต่อมาห่วงจ๋...,1.0
9,เชฟแด๊ดดี๊ ฮืออ หลง #MasterChefAllStarsThailand,1.0
